# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('../output_data/output_weather_data.csv')

weather_df


,City_ID,City,Country,id,lat,lon,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),dt
0,0,Ushuaia,AR,3833367,-54.8000,-68.3000,37.06,75,0,3.44,08/22/2022
1,1,Ocós,GT,3592086,14.5094,-92.1933,75.38,74,64,4.47,08/22/2022
2,2,Leningradskiy,RU,2123814,69.3833,178.4167,38.08,83,0,13.02,08/22/2022
3,3,Arraial do Cabo,BR,3471451,-22.9661,-42.0278,71.46,64,40,4.61,08/22/2022
4,4,Ancud,CL,3899695,-41.8697,-73.8203,46.51,90,100,3.80,08/22/2022
...,...,...,...,...,...,...,...,...,...,...,...
561,561,Uthai Thani,TH,1149965,15.3833,99.5500,78.37,78,100,4.63,08/22/2022
562,562,Visnes,NO,3131500,59.3556,5.2242,62.98,59,20,9.22,08/22/2022
563,563,Plettenberg Bay,ZA,964712,-34.0527,23.3716,59.41,71,85,6.29,08/22/2022
564,564,Atar,MR,2381334,20.5169,-13.0499,99.39,26,89,1.25,08/22/2022


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Use lat and lon as locations
# Use humidity as weight
locations = weather_df[['lat', 'lon']]
weight = weather_df['Humidity (%)'].astype(float)


In [5]:
# Create and add the heatmap layer to the map
fig = gmaps.figure()
cities_heatmap_layer = gmaps.heatmap_layer(locations, weight, 
                                           max_intensity = 100, point_radius = 5)
fig.add_layer(cities_heatmap_layer)

fig


Figure(layout=FigureLayout(height='420px'))

In [6]:
# plt.savefig('../output_data/heatmap.png')

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
# New Dataframe with ideal weather
# Temperature between 50F and 65F
# Humidity under 90%
# Wind Speed under 15 mph
# No preference for cloud cover
vacation_cities = weather_df.loc[(weather_df['Max Temp (F)'] > 50) & 
                                 (weather_df['Max Temp (F)'] < 65) &
                                 (weather_df['Humidity (%)'] < 90) &
                                 (weather_df['Wind Speed (mph)'] < 15)]
vacation_cities = vacation_cities.reset_index(drop = True)
vacation_cities

,City_ID,City,Country,id,lat,lon,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),dt
0,5,Okha,RU,2122614,53.5739,142.9478,54.01,84,15,7.40,08/22/2022
1,10,Mackay,AU,2159220,-21.1500,149.2000,64.69,70,100,10.07,08/22/2022
2,21,Esso,RU,2125711,55.9333,158.7000,53.85,86,100,4.65,08/22/2022
3,39,Port Alfred,ZA,964432,-33.5906,26.8910,64.18,70,89,5.01,08/22/2022
4,49,East London,ZA,1006984,-33.0153,27.9116,63.90,65,0,8.34,08/22/2022
...,...,...,...,...,...,...,...,...,...,...,...
64,512,Chapais,CA,5919850,49.7834,-74.8492,62.55,66,16,4.65,08/22/2022
65,517,Mandalgovi,MN,2030065,45.7625,106.2708,54.84,40,53,8.50,08/22/2022
66,548,Urengoy,RU,1488414,65.9667,78.3667,60.04,76,100,6.04,08/22/2022
67,562,Visnes,NO,3131500,59.3556,5.2242,62.98,59,20,9.22,08/22/2022


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
# Create hotel_df to hold results and add a "Hotel Name" Column
hotel_df = vacation_cities
hotel_df['Hotel Name'] = ''

# Create a list to hold hotel names
hotels = []

In [16]:
# Set parameters to search for hotels within 5000 meters
# Make API call for coordinates and append results into dataframe

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


for city in range(len(hotel_df)):
    lat = hotel_df.loc[city]['lat']
    lon = hotel_df.loc[city]['lon']
    
    params = {'location': f"{lat},{lon}",
          'radius':5000,
          'type':'lodging',
          'key':g_key}
    
    response = requests.get(url, params = params)
    response_json = response.json()
    try:
        hotels.append(response_json['results'][0]['name'])
    except:
        hotels.append('No Results')

hotel_df['Hotel Name'] = hotels
hotel_df
    


,City_ID,City,Country,id,lat,lon,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),dt,Hotel Name
0,5,Okha,RU,2122614,53.5739,142.9478,54.01,84,15,7.40,08/22/2022,Sakhalin-Sfera
1,10,Mackay,AU,2159220,-21.1500,149.2000,64.69,70,100,10.07,08/22/2022,International Lodge Motel
2,21,Esso,RU,2125711,55.9333,158.7000,53.85,86,100,4.65,08/22/2022,Paramushir Tur
3,39,Port Alfred,ZA,964432,-33.5906,26.8910,64.18,70,89,5.01,08/22/2022,The Halyards Hotel
4,49,East London,ZA,1006984,-33.0153,27.9116,63.90,65,0,8.34,08/22/2022,Tu Casa
...,...,...,...,...,...,...,...,...,...,...,...,...
64,512,Chapais,CA,5919850,49.7834,-74.8492,62.55,66,16,4.65,08/22/2022,Hôtel Opémiska
65,517,Mandalgovi,MN,2030065,45.7625,106.2708,54.84,40,53,8.50,08/22/2022,В.Ганбат
66,548,Urengoy,RU,1488414,65.9667,78.3667,60.04,76,100,6.04,08/22/2022,Gostinnitsa Medveditsa
67,562,Visnes,NO,3131500,59.3556,5.2242,62.98,59,20,9.22,08/22/2022,Park Inn Haugesund Airport


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [18]:
# Add marker layer ontop of heat map
hotel_markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(hotel_markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))